In [1]:
from google.colab import drive
drive.mount('gdrive')

Mounted at gdrive


In [2]:
cd /content/gdrive/MyDrive/College/Semester5/NLP/project

/content/gdrive/MyDrive/College/Semester5/NLP/project


In [3]:
from tensorflow.keras import backend as K
from tensorflow.keras import initializers, regularizers, constraints
from tensorflow.keras.layers import Layer


class Attention(Layer):
    def __init__(self, step_dim,
                 W_regularizer=None, b_regularizer=None,
                 W_constraint=None, b_constraint=None,
                 bias=True, **kwargs):
        """
        Keras Layer that implements an Attention mechanism for temporal data.
        Supports Masking.
        Follows the work of Raffel et al. [https://arxiv.org/abs/1512.08756]
        # Input shape
            3D tensor with shape: `(samples, steps, features)`.
        # Output shape
            2D tensor with shape: `(samples, features)`.
        :param kwargs:
        Just put it on top of an RNN Layer (GRU/LSTM/SimpleRNN) with return_sequences=True.
        The dimensions are inferred based on the output shape of the RNN.
        Example:
            # 1
            model.add(LSTM(64, return_sequences=True))
            model.add(Attention())
            # next add a Dense layer (for classification/regression) or whatever...
            # 2
            hidden = LSTM(64, return_sequences=True)(words)
            sentence = Attention()(hidden)
            # next add a Dense layer (for classification/regression) or whatever...
        """
        self.supports_masking = True
        self.init = initializers.get('glorot_uniform')

        self.W_regularizer = regularizers.get(W_regularizer)
        self.b_regularizer = regularizers.get(b_regularizer)

        self.W_constraint = constraints.get(W_constraint)
        self.b_constraint = constraints.get(b_constraint)

        self.bias = bias
        self.step_dim = step_dim
        self.features_dim = 0

        super(Attention, self).__init__(**kwargs)

    def build(self, input_shape):
        assert len(input_shape) == 3

        self.W = self.add_weight(name='{}_W'.format(self.name),
                                 shape=(input_shape[-1],),
                                 initializer=self.init,
                                 regularizer=self.W_regularizer,
                                 constraint=self.W_constraint)
        self.features_dim = input_shape[-1]

        if self.bias:
            self.b = self.add_weight(name='{}_b'.format(self.name),
                                     shape=(input_shape[1],),
                                     initializer='zero',
                                     regularizer=self.b_regularizer,
                                     constraint=self.b_constraint)
        else:
            self.b = None

        self.built = True

    def compute_mask(self, input, input_mask=None):
        # do not pass the mask to the next layers
        return None

    def call(self, x, mask=None):
        features_dim = self.features_dim
        step_dim = self.step_dim

        e = K.reshape(K.dot(K.reshape(x, (-1, features_dim)), K.reshape(self.W, (features_dim, 1))), (-1, step_dim))  # e = K.dot(x, self.W)
        if self.bias:
            e += self.b
        e = K.tanh(e)

        a = K.exp(e)
        # apply mask after the exp. will be re-normalized next
        if mask is not None:
            # cast the mask to floatX to avoid float64 upcasting in theano
            a *= K.cast(mask, K.floatx())
        # in some cases especially in the early stages of training the sum may be almost zero
        # and this results in NaN's. A workaround is to add a very small positive number ε to the sum.
        a /= K.cast(K.sum(a, axis=1, keepdims=True) + K.epsilon(), K.floatx())
        a = K.expand_dims(a)

        c = K.sum(a * x, axis=1)
        return c

    def compute_output_shape(self, input_shape):
        return input_shape[0], self.features_dim


In [4]:
from tensorflow.keras import Input, Model
from tensorflow.keras.layers import Embedding, Dense, Bidirectional, TimeDistributed
# from keras.layers import CuDNNGRU
from keras.layers import GRU

class HCAN(Model):
    def __init__(self,
                 maxlen_sentence,
                 maxlen_word,
                 max_features,
                 embedding_dims,
                 class_num=1,
                 last_activation='sigmoid'):
        super(HCAN, self).__init__()
        self.maxlen_sentence = maxlen_sentence
        self.maxlen_word = maxlen_word
        self.max_features = max_features
        self.embedding_dims = embedding_dims
        self.class_num = class_num
        self.last_activation = last_activation
        # Word part
        input_word = Input(shape=(self.maxlen_word,))
        x_word = Embedding(self.max_features, self.embedding_dims, input_length=self.maxlen_word)(input_word)
        x_word = layers.Convolution1D(100, 10, activation="relu", padding = 'same')(x_word)
        # x_word = layers.GlobalMaxPool1D()(x_word)

        x_word = Bidirectional(GRU(128, return_sequences=True))(x_word)  # LSTM or GRU
        x_word = Attention(self.maxlen_word)(x_word)
        model_word = Model(input_word, x_word)
        # Sentence part
        self.word_encoder_att = TimeDistributed(model_word)
        self.sentence_encoder = Bidirectional(GRU(128, return_sequences=True))  # LSTM or GRU
        self.sentence_att = Attention(self.maxlen_sentence)
        # Output part
        self.classifier = Dense(self.class_num, activation=self.last_activation)

    def call(self, inputs):
        if len(inputs.get_shape()) != 3:
            raise ValueError('The rank of inputs of HAN must be 3, but now is %d' % len(inputs.get_shape()))
        if inputs.get_shape()[1] != self.maxlen_sentence:
            raise ValueError('The maxlen_sentence of inputs of HAN must be %d, but now is %d' % (self.maxlen_sentence, inputs.get_shape()[1]))
        if inputs.get_shape()[2] != self.maxlen_word:
            raise ValueError('The maxlen_word of inputs of HAN must be %d, but now is %d' % (self.maxlen_word, inputs.get_shape()[2]))
        x_sentence = self.word_encoder_att(inputs)
        x_sentence = self.sentence_encoder(x_sentence)
        x_sentence = self.sentence_att(x_sentence)
        output = self.classifier(x_sentence)
        return output


In [5]:
from tensorflow.keras import layers
from keras.callbacks import ModelCheckpoint

## **Deploying from Colab**

In [6]:
cd /content/gdrive/MyDrive/College/Semester5/NLP/project/Final

/content/gdrive/MyDrive/College/Semester5/NLP/project/Final


In [10]:
!pip install flask_ngrok

In [11]:

import pandas as pd
import pickle
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
import joblib
from tensorflow.keras import Input, Model
from tensorflow.keras.layers import Embedding, Dense, Bidirectional, TimeDistributed
from keras.layers import GRU
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing import sequence
import pandas as pd
import numpy
from keras.preprocessing import text, sequence
from sklearn import metrics
from tensorflow.keras import layers
from keras.callbacks import ModelCheckpoint
from tensorflow.keras import backend as K
from tensorflow.keras import initializers, regularizers, constraints
from tensorflow.keras.layers import Layer
import os
import sys
import joblib

In [16]:
import flask
from flask import Flask,render_template,url_for,request
import joblib
from flask_ngrok import run_with_ngrok
app = Flask(__name__)
run_with_ngrok(app)
import os
@app.route('/')
def home():
  return render_template('home.html')

@app.route('/predict',methods=['POST'])
def predict():
  max_features = 5000
  maxlen_sentence = 16
  maxlen_word = 25
  batch_size = 32
  embedding_dims = 50
  cwd = os.getcwd()
  print(cwd)
  model2 = HCAN(maxlen_sentence, maxlen_word, max_features, embedding_dims)
  model2.compile('adam', 'binary_crossentropy', metrics=['accuracy'])
  token = joblib.load("token.sav")
  model2.load_weights('hcan_final_run')
  if request.method == 'POST':
    message = request.form['message']
    data = numpy.array([message])
    data = sequence.pad_sequences(token.texts_to_sequences(data), maxlen=maxlen_sentence * maxlen_word)
    data = data.reshape((len(data), maxlen_sentence, maxlen_word))
    my_prediction = model2.predict(data)

  if my_prediction[0][0] > 0.5:
    preds = 1
  else:
    preds = 0
  return render_template('result.html', prediction = preds)
 
app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://5b3e-35-184-80-233.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [28/Nov/2021 01:24:40] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [28/Nov/2021 01:24:41] "GET /static/static/styles.css HTTP/1.1" 404 -
127.0.0.1 - - [28/Nov/2021 01:24:41] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [28/Nov/2021 01:25:09] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [28/Nov/2021 01:25:09] "GET /static/static/styles.css HTTP/1.1" 404 -


/content/gdrive/My Drive/College/Semester5/NLP/project/Final


127.0.0.1 - - [28/Nov/2021 01:25:45] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [28/Nov/2021 01:25:45] "GET /static/css/styles.css HTTP/1.1" 404 -


/content/gdrive/My Drive/College/Semester5/NLP/project/Final


127.0.0.1 - - [28/Nov/2021 01:27:34] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [28/Nov/2021 01:27:34] "GET /static/css/styles.css HTTP/1.1" 404 -
